In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2
import folium

In [2]:
from update_vars import analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH

In [3]:
import sjoin_stops_to_segments

In [4]:
analysis_date

'2024-09-18'

In [5]:
# (1) Aggregate stop times - by stop_id, find max trips in AM/PM peak
# takes 1 min
max_arrivals_by_stop = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
).pipe(sjoin_stops_to_segments.prep_stop_times).pipe(sjoin_stops_to_segments.stop_times_aggregation_max_by_stop, analysis_date)

In [6]:
# (1) Aggregate stop times - by stop_id, find max trips in AM/PM peak
# takes 1 min
max_arrivals_by_stop_single = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
).pipe(sjoin_stops_to_segments.prep_stop_times).pipe(
    sjoin_stops_to_segments.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=True)

In [7]:
new_hq_multi = max_arrivals_by_stop >> filter(_.am_max_trips_hr > 4, _.pm_max_trips_hr > 4) #  new HQ corridor (still multi-route)
new_ms_multi = max_arrivals_by_stop >> filter(_.am_max_trips_hr > 3, _.pm_max_trips_hr > 3) #  new major stop precursor (still multi-route)
new_hq_single = max_arrivals_by_stop_single >> filter(_.am_max_trips_hr > 4, _.pm_max_trips_hr > 4) #  new HQ corridor (single-route)
new_ms_single = max_arrivals_by_stop_single >> filter(_.am_max_trips_hr > 3, _.pm_max_trips_hr > 3) #  new major stop precursor (single-route)

## preliminary mapping

* add bus/rail/ferry MTS in orange to help decide

In [8]:
OLD_EXPORT_PATH = f"{GCS_FILE_PATH}export/{rt_dates.DATES['aug2024']}/"

In [9]:
old_hq_points = gpd.read_parquet(f'{OLD_EXPORT_PATH}ca_hq_transit_stops.parquet')

In [10]:
non_bus = old_hq_points >> filter(-_.hqta_type.isin(['hq_corridor_bus', 'major_stop_bus']))

In [11]:
feeds = (gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_date)
         >> select(_.feed_key, _.schedule_gtfs_dataset_key == _.gtfs_dataset_key)
         >> filter(_.schedule_gtfs_dataset_key.isin(max_arrivals_by_stop.schedule_gtfs_dataset_key))
        )

In [12]:
stops = gtfs_utils_v2.get_stops(selected_date=analysis_date, operator_feeds=feeds.feed_key,
                       stop_cols=['feed_key', 'stop_id'])

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [13]:
stops = stops >> inner_join(_, feeds, on='feed_key')

In [14]:
def add_to_map(test_df, m=None, **kwargs):
    gdf = stops >> inner_join(_, test_df, on=['schedule_gtfs_dataset_key', 'stop_id'])
    m = gdf.explore(m = m, **kwargs)
    return m

In [15]:
m1 = old_hq_points.explore(color='blue')
m1 = non_bus.explore(m = m, color='orange')

NameError: name 'm' is not defined

In [ ]:
m1 = add_to_map(new_hq_multi, m=m1, color='green')

In [ ]:
m1 = add_to_map(new_hq_single, m=m1, color='red')

In [ ]:
folium.LayerControl().add_to(m1);

In [ ]:
m1

## major stops

In [ ]:
m = old_hq_points.explore(color='blue')
m = non_bus.explore(m = m, color='orange')

In [ ]:
m = add_to_map(new_ms_multi, m=m, color='green')

In [ ]:
m = add_to_map(new_ms_single, m=m, color='red')

In [ ]:
folium.LayerControl().add_to(m);

In [ ]:
m

# next step: intersections